# PSO-LSTM による USD/JPY 1本先対数収益予測（5分足）

設計書に基づくパイプライン：
- **目的変数**: 1本先の対数収益 $y_t = \ln(P_{t+1}/P_t)$
- **データ**: data/ の CSV（5分足）
- **前処理**: 対数比率+MODWT(Haar, level 3)、テクニカル指標（終値との対数乖離）、曜日・時間の sin/cos エンコーディング
- **スケーリング**: EWMA ローリング標準化、±3σ クリップ（訓練データのみでパラメータ算出）
- **評価**: RMSE, MAE, MAPE(%), R2

In [ ]:
from pso_lstm_common import (
    BATCH_SIZE,
    NEURON_BOUNDS,
    EPOCH_BOUNDS,
    LAYER_BOUNDS,
    PSO_W,
    PSO_C1,
    PSO_C2,
    PSO_PARTICLES,
    PSO_ITERS,
    build_lstm_model,
    compute_metrics,
    create_sequences,
    load_csv,
    preprocess_5m_pipeline,
    pso_optimize,
    remove_high_corr_features,
    resample_ohlcv,
    scale_ewma_train_val_test,
    train_val_test_split,
)

In [ ]:
import warnings
import numpy as np
import tensorflow as tf

warnings.filterwarnings("ignore")
np.random.seed(42)
tf.random.set_seed(42)

physical_gpus = tf.config.list_physical_devices("GPU")
if physical_gpus:
    for gpu in physical_gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    strategy = tf.distribute.MirroredStrategy() if len(physical_gpus) > 1 else None
else:
    strategy = None

In [ ]:
# ===== 設定（設計書付録） =====
DATA_PATH = "data/merged-usdjpy-base-2024-03-01-2025-10-31-5m.csv"
LOOKBACK = 100
TRAIN_RATIO = 0.8
VAL_RATIO = 0.2
CORR_THRESHOLD = 0.95
TARGET_COL = "target_log_return"

## 1. データ読込・前処理

In [ ]:
df_raw = load_csv(DATA_PATH)
df_raw = resample_ohlcv(df_raw, minutes=5)
df = preprocess_5m_pipeline(df_raw)
print(df.shape)
df.head(2)

In [ ]:
# 説明変数: 設計書の列のみ（生価格・target は除外）
exclude_from_features = {
    "target_log_return", "open", "high", "low", "close", "volume", "vwap",
    "EURUSD_close", "EURJPY_close",
}
feature_cols = [c for c in df.columns if c not in exclude_from_features]
df_work = df[feature_cols + [TARGET_COL]].copy()
df_work = df_work.dropna()
print("dropna 後:", df_work.shape)

In [ ]:
# 特徴量削減: 目的変数との絶対相関が閾値超の列を削除
df_work, dropped = remove_high_corr_features(
    df_work, target_col=TARGET_COL, threshold=CORR_THRESHOLD
)
feature_cols = [c for c in df_work.columns if c != TARGET_COL]
print("削除した列:", dropped)
print("説明変数数:", len(feature_cols))

## 2. シーケンス作成・分割・スケーリング

In [ ]:
features = df_work[feature_cols].values
target = df_work[TARGET_COL].values
# 可視化用に終値（価格スケール復元用）を保持。前処理前の close を同じインデックスで取得
close_series = df_raw.reindex(df_work.index).ffill()["close"]

X, y, last_close = create_sequences(
    features, target, lookback=LOOKBACK, close_series=close_series
)
print("X:", X.shape, "y:", y.shape, "last_close:", last_close.shape)

In [ ]:
(
    X_train, y_train, X_val, y_val, X_test, y_test,
    lc_train, lc_val, lc_test,
) = train_val_test_split(X, y, train_ratio=TRAIN_RATIO, val_ratio=VAL_RATIO, last_close=last_close)
print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

In [ ]:
X_train_s, X_val_s, X_test_s, ewma_scaler = scale_ewma_train_val_test(
    X_train, X_val, X_test, feature_names=feature_cols
)
print("スケーリング完了")

## 3. PSO でハイパーパラメータ探索

In [ ]:
input_shape = (LOOKBACK, len(feature_cols))
best_pos, best_cost = pso_optimize(
    X_train_s, y_train, X_val_s, y_val, input_shape,
    csv_log_path="pso_training.csv",
    strategy=strategy,
)
units = int(round(best_pos[0]))
epochs = int(round(best_pos[1]))
n_layers = int(round(best_pos[2]))
print(f"PSO 最良: units={units}, epochs={epochs}, layers={n_layers}, 検証 RMSE={best_cost:.6f}")

In [ ]:
# オプション: モデルとテスト結果を保存（可視化ノートブックで利用）
# final_model.save("best_model_pso_lstm_5m.keras")
# np.savez("test_result.npz", y_test=y_test, y_pred=y_pred, lc_test=lc_test)

## 4. 最良パラメータで最終モデル学習・テスト評価

In [ ]:
tf.keras.backend.clear_session()
if strategy is not None:
    with strategy.scope():
        final_model = build_lstm_model(input_shape, n_layers, units)
else:
    final_model = build_lstm_model(input_shape, n_layers, units)

final_model.fit(
    X_train_s, y_train,
    validation_data=(X_val_s, y_val),
    epochs=epochs,
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=5, restore_best_weights=True
        ),
        tf.keras.callbacks.CSVLogger("final_log_pso_lstm_5m.csv"),
    ],
    verbose=1,
)

In [ ]:
y_pred = final_model.predict(X_test_s, verbose=0).ravel()
rmse, mae, mape, r2 = compute_metrics(y_test, y_pred)
print(f"テスト RMSE={rmse:.6f}, MAE={mae:.6f}, MAPE={mape:.4f}%, R2={r2:.4f}")

In [ ]:
# 価格スケールで復元（last_close * exp(pred_log_return)）
pred_close = lc_test * np.exp(y_pred)
actual_close = lc_test * np.exp(y_test)
print("予測終値と実測終値のサンプル（先頭5件）:")
print("actual:", actual_close[:5])
print("pred:  ", pred_close[:5])